In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import gaussian_kde
import numpy as np

plt.style.use('ggplot')

# Data

In [2]:
data = pd.read_csv('train.csv', ',', keep_default_na=False)

data.info

C:\Users\yiche\AppData\Local\Temp\ipykernel_17396\517929564.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  data = pd.read_csv('train.csv', ',', keep_default_na=False)


<bound method DataFrame.info of         Id  MSSubClass MSZoning LotFrontage  LotArea Street Alley LotShape  \
0        1          60       RL        65.0     8450   Pave            Reg   
1        2          20       RL        80.0     9600   Pave            Reg   
2        3          60       RL        68.0    11250   Pave            IR1   
3        4          70       RL        60.0     9550   Pave            IR1   
4        5          60       RL        84.0    14260   Pave            IR1   
...    ...         ...      ...         ...      ...    ...   ...      ...   
1455  1456          60       RL        62.0     7917   Pave            Reg   
1456  1457          20       RL        85.0    13175   Pave            Reg   
1457  1458          70       RL        66.0     9042   Pave            Reg   
1458  1459          20       RL        68.0     9717   Pave            Reg   
1459  1460          20       RL        75.0     9937   Pave            Reg   

     LandContour Utilities  ...

# Data Cleaning

## Add PriceType

In [3]:
data['PriceType'] = np.where(
                            data['SalePrice'] >= 300000,
                            'HIGH',
                            np.where(
                                data['SalePrice'] <= 150000,
                                'LOW',
                                'MEDIUM'
                                )
                            )

In [4]:
data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,PriceType
0,1,60,RL,65.0,8450,Pave,,Reg,Lvl,AllPub,...,,,,0,2,2008,WD,Normal,208500,MEDIUM
1,2,20,RL,80.0,9600,Pave,,Reg,Lvl,AllPub,...,,,,0,5,2007,WD,Normal,181500,MEDIUM
2,3,60,RL,68.0,11250,Pave,,IR1,Lvl,AllPub,...,,,,0,9,2008,WD,Normal,223500,MEDIUM
3,4,70,RL,60.0,9550,Pave,,IR1,Lvl,AllPub,...,,,,0,2,2006,WD,Abnorml,140000,LOW
4,5,60,RL,84.0,14260,Pave,,IR1,Lvl,AllPub,...,,,,0,12,2008,WD,Normal,250000,MEDIUM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,,Reg,Lvl,AllPub,...,,,,0,8,2007,WD,Normal,175000,MEDIUM
1456,1457,20,RL,85.0,13175,Pave,,Reg,Lvl,AllPub,...,,MnPrv,,0,2,2010,WD,Normal,210000,MEDIUM
1457,1458,70,RL,66.0,9042,Pave,,Reg,Lvl,AllPub,...,,GdPrv,Shed,2500,5,2010,WD,Normal,266500,MEDIUM
1458,1459,20,RL,68.0,9717,Pave,,Reg,Lvl,AllPub,...,,,,0,4,2010,WD,Normal,142125,LOW


## Rm value in col with real missing values

from all the columns, the following are columns with real missing values:
- LotFrontage
- MasVnrType
- MasVnrArea
- Electrical
- GarageYrBlt

### Apply nan to certain columns

In [5]:
columns = ['LotFrontage', 'MasVnrType', 'MasVnrArea', 'Electrical', 'GarageYrBlt']

for column in columns:
    data[column] = data[column].replace('NA', np.nan) 

In [6]:
data.info

<bound method DataFrame.info of         Id  MSSubClass MSZoning LotFrontage  LotArea Street Alley LotShape  \
0        1          60       RL        65.0     8450   Pave            Reg   
1        2          20       RL        80.0     9600   Pave            Reg   
2        3          60       RL        68.0    11250   Pave            IR1   
3        4          70       RL        60.0     9550   Pave            IR1   
4        5          60       RL        84.0    14260   Pave            IR1   
...    ...         ...      ...         ...      ...    ...   ...      ...   
1455  1456          60       RL        62.0     7917   Pave            Reg   
1456  1457          20       RL        85.0    13175   Pave            Reg   
1457  1458          70       RL        66.0     9042   Pave            Reg   
1458  1459          20       RL        68.0     9717   Pave            Reg   
1459  1460          20       RL        75.0     9937   Pave            Reg   

     LandContour Utilities  ...

### Check which col have true missing value

In [7]:
data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,PriceType
0,1,60,RL,65.0,8450,Pave,,Reg,Lvl,AllPub,...,,,,0,2,2008,WD,Normal,208500,MEDIUM
1,2,20,RL,80.0,9600,Pave,,Reg,Lvl,AllPub,...,,,,0,5,2007,WD,Normal,181500,MEDIUM
2,3,60,RL,68.0,11250,Pave,,IR1,Lvl,AllPub,...,,,,0,9,2008,WD,Normal,223500,MEDIUM
3,4,70,RL,60.0,9550,Pave,,IR1,Lvl,AllPub,...,,,,0,2,2006,WD,Abnorml,140000,LOW
4,5,60,RL,84.0,14260,Pave,,IR1,Lvl,AllPub,...,,,,0,12,2008,WD,Normal,250000,MEDIUM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,,Reg,Lvl,AllPub,...,,,,0,8,2007,WD,Normal,175000,MEDIUM
1456,1457,20,RL,85.0,13175,Pave,,Reg,Lvl,AllPub,...,,MnPrv,,0,2,2010,WD,Normal,210000,MEDIUM
1457,1458,70,RL,66.0,9042,Pave,,Reg,Lvl,AllPub,...,,GdPrv,Shed,2500,5,2010,WD,Normal,266500,MEDIUM
1458,1459,20,RL,68.0,9717,Pave,,Reg,Lvl,AllPub,...,,,,0,4,2010,WD,Normal,142125,LOW


### Remove the missing lines

In [8]:
data = data.dropna()

data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,PriceType
0,1,60,RL,65.0,8450,Pave,,Reg,Lvl,AllPub,...,,,,0,2,2008,WD,Normal,208500,MEDIUM
1,2,20,RL,80.0,9600,Pave,,Reg,Lvl,AllPub,...,,,,0,5,2007,WD,Normal,181500,MEDIUM
2,3,60,RL,68.0,11250,Pave,,IR1,Lvl,AllPub,...,,,,0,9,2008,WD,Normal,223500,MEDIUM
3,4,70,RL,60.0,9550,Pave,,IR1,Lvl,AllPub,...,,,,0,2,2006,WD,Abnorml,140000,LOW
4,5,60,RL,84.0,14260,Pave,,IR1,Lvl,AllPub,...,,,,0,12,2008,WD,Normal,250000,MEDIUM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,,Reg,Lvl,AllPub,...,,,,0,8,2007,WD,Normal,175000,MEDIUM
1456,1457,20,RL,85.0,13175,Pave,,Reg,Lvl,AllPub,...,,MnPrv,,0,2,2010,WD,Normal,210000,MEDIUM
1457,1458,70,RL,66.0,9042,Pave,,Reg,Lvl,AllPub,...,,GdPrv,Shed,2500,5,2010,WD,Normal,266500,MEDIUM
1458,1459,20,RL,68.0,9717,Pave,,Reg,Lvl,AllPub,...,,,,0,4,2010,WD,Normal,142125,LOW


# Remove the SalePrice column

In [9]:
data = data.drop(['SalePrice'], axis=1)

In [10]:
data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,PriceType
0,1,60,RL,65.0,8450,Pave,,Reg,Lvl,AllPub,...,0,,,,0,2,2008,WD,Normal,MEDIUM
1,2,20,RL,80.0,9600,Pave,,Reg,Lvl,AllPub,...,0,,,,0,5,2007,WD,Normal,MEDIUM
2,3,60,RL,68.0,11250,Pave,,IR1,Lvl,AllPub,...,0,,,,0,9,2008,WD,Normal,MEDIUM
3,4,70,RL,60.0,9550,Pave,,IR1,Lvl,AllPub,...,0,,,,0,2,2006,WD,Abnorml,LOW
4,5,60,RL,84.0,14260,Pave,,IR1,Lvl,AllPub,...,0,,,,0,12,2008,WD,Normal,MEDIUM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,,Reg,Lvl,AllPub,...,0,,,,0,8,2007,WD,Normal,MEDIUM
1456,1457,20,RL,85.0,13175,Pave,,Reg,Lvl,AllPub,...,0,,MnPrv,,0,2,2010,WD,Normal,MEDIUM
1457,1458,70,RL,66.0,9042,Pave,,Reg,Lvl,AllPub,...,0,,GdPrv,Shed,2500,5,2010,WD,Normal,MEDIUM
1458,1459,20,RL,68.0,9717,Pave,,Reg,Lvl,AllPub,...,0,,,,0,4,2010,WD,Normal,LOW


# Remove the ID column

In [11]:
data = data.drop(['Id'], axis=1)

data

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,PriceType
0,60,RL,65.0,8450,Pave,,Reg,Lvl,AllPub,Inside,...,0,,,,0,2,2008,WD,Normal,MEDIUM
1,20,RL,80.0,9600,Pave,,Reg,Lvl,AllPub,FR2,...,0,,,,0,5,2007,WD,Normal,MEDIUM
2,60,RL,68.0,11250,Pave,,IR1,Lvl,AllPub,Inside,...,0,,,,0,9,2008,WD,Normal,MEDIUM
3,70,RL,60.0,9550,Pave,,IR1,Lvl,AllPub,Corner,...,0,,,,0,2,2006,WD,Abnorml,LOW
4,60,RL,84.0,14260,Pave,,IR1,Lvl,AllPub,FR2,...,0,,,,0,12,2008,WD,Normal,MEDIUM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,,Reg,Lvl,AllPub,Inside,...,0,,,,0,8,2007,WD,Normal,MEDIUM
1456,20,RL,85.0,13175,Pave,,Reg,Lvl,AllPub,Inside,...,0,,MnPrv,,0,2,2010,WD,Normal,MEDIUM
1457,70,RL,66.0,9042,Pave,,Reg,Lvl,AllPub,Inside,...,0,,GdPrv,Shed,2500,5,2010,WD,Normal,MEDIUM
1458,20,RL,68.0,9717,Pave,,Reg,Lvl,AllPub,Inside,...,0,,,,0,4,2010,WD,Normal,LOW


# Save the cleaned data

In [12]:
data.to_csv('clean_data.csv', index=False)